In [5]:
import numpy as np
#import mayavi.mlab as mlab
import sys
sys.path.append('/home/sbreuers/vathos/s4g-release/inference/')
from grasp_proposal.utils.grasp_visualizer import GraspVisualizer
import open3d
from grasp_proposal.cloud_processor.cloud_processor import CloudPreProcessor
import torch
import pickle

#from grasp_proposal.grasp_proposal_test import load_static_data_batch
def load_static_data_batch():
    #single_training_data = np.load("inference/2638_view_0.p", allow_pickle=True)
    single_training_data = np.load("inference/last_pointcloud_big.p", allow_pickle=True)
    cloud_array = single_training_data["point_cloud"]
    cloud = CloudPreProcessor(open3d.geometry.PointCloud(open3d.utility.Vector3dVector(cloud_array.T)), False)

    # do not filter workspace here since training data
    cloud.voxelize()
    cloud.remove_outliers()
    points = np.asarray(cloud.pcd.points)
    if points.shape[0] > 25600:
        random_index = np.random.choice(np.arange(points.shape[0]), 25600, replace=False)
    else:
        random_index = np.random.choice(np.arange(points.shape[0]), 25600, replace=True)

    points = points[random_index, :]
    data_batch = {"scene_points": torch.tensor(points, dtype=torch.float32).unsqueeze(0).transpose(1, 2)}
    return data_batch, cloud.pcd

In [ ]:
 # visualize result
data_batch, pcd = load_static_data_batch()
top_poses = np.load("inference/grasp_proposal/top_frames.npy")
visualizer = GraspVisualizer(pcd)
visualizer.add_multiple_poses(top_poses)
visualizer.visualize()

In [3]:
# inspect their pointcloud
single_training_data = np.load("inference/2638_view_0.p", allow_pickle=True)
print(single_training_data.keys())
print("point cloud")
print(single_training_data["point_cloud"].shape)
print(single_training_data["point_cloud"][:,0:5])
print("search score")
print(single_training_data["search_score"].shape)
print(single_training_data["search_score"][0:5])
print("antipodal_score")
print(single_training_data["antipodal_score"].shape)
print(single_training_data["antipodal_score"][0:5])
print("valid_frame")
print(single_training_data["valid_frame"].shape)
print(single_training_data["valid_frame"][0,:,:])
print("valid_index")
print(single_training_data["valid_index"].shape)
print(single_training_data["valid_index"][0:5])
print("objects_label")
print(single_training_data["objects_label"].shape)
print(single_training_data["objects_label"][0:5])


dict_keys(['search_score', 'antipodal_score', 'valid_frame', 'valid_index', 'point_cloud', 'objects_label'])
point cloud
(3, 48902)
[[ 0.23269033  0.22019997  0.22615957  0.19583142  0.1853585 ]
 [ 0.34287396  0.3386026   0.33791447  0.34177426  0.34445158]
 [-1.014849   -1.0107541  -1.0115755  -1.0424486  -1.0506148 ]]
search score
(306,)
[456.6667  387.66666 316.      294.      462.     ]
antipodal_score
(306,)
[0.97540945 0.97183985 0.98422176 0.97475517 0.9976949 ]
valid_frame
(306, 4, 4)
[[-0.21604782 -0.19880381  0.95592916 -0.24223596]
 [-0.8274021   0.55708593 -0.07114293  0.2534126 ]
 [-0.5183912  -0.80630785 -0.28484792 -1.2580454 ]
 [ 0.          0.          0.          1.        ]]
valid_index
(306,)
[2389 2928 3564 3687 3688]
objects_label
(306,)
[105 105  70  70  70]


In [ ]:
# save our pointcloud
our_pointcloud = np.load("/home/sbreuers/data/images/last_pointcloud_big.npy").T
our_pointcloud[0,:] *=  -1
our_pointcloud[2,:] *=  -1
i = our_pointcloud[0,:].copy()
our_pointcloud[0,:] = our_pointcloud[1,:].copy()
our_pointcloud[1,:] = i
print(our_pointcloud.shape)
pointcloud_dict = {}
pointcloud_dict["point_cloud"] = our_pointcloud
output_path = "inference/last_pointcloud_big.p"
with open(output_path, 'wb') as file:
    pickle.dump(pointcloud_dict, file)

(3, 217088)


In [ ]:
# inspect our pointcloud
single_training_data = np.load("inference/last_pointcloud_10.p", allow_pickle=True)
print(single_training_data.keys())
print(single_training_data["point_cloud"].shape)
print(single_training_data["point_cloud"][:,0:5])

dict_keys(['point_cloud'])
(3, 217088)
[[-0.47898031 -0.47906378 -0.47914478 -0.47922591 -0.47930944]
 [ 0.61004786  0.60781444  0.60557707  0.60333909  0.6011033 ]
 [-0.8605808  -0.86073077 -0.86087632 -0.86102206 -0.86117214]]


In [ ]:
point_cloud = open3d.io.read_point_cloud("inference/grasp_proposal/output/test_step00000/pred_pts.ply")
open3d.visualization.draw_geometries([point_cloud])

In [1]:
# visualize grasps
import os

import numpy as np
import open3d
from matplotlib import cm

import sys
sys.path.append('/home/sbreuers/vathos/s4g-release/data_gen/')

color_map = cm.get_cmap('jet', 256)
data_dir = "/home/sbreuers/vathos/s4g-release/objects/processed_single_object_grasp" 
object_names = ["Lager"]
for object_name in object_names:
    data_path = os.path.join(data_dir, '{}.pkl'.format(object_name))

    data = np.load(data_path, allow_pickle=True)
    cloud = data['cloud']
    grasp_pose = data['grasp_pose']
    grasp_point_index = data['grasp_point_index'].astype(int)
    grasp_point_index = grasp_point_index.tolist()
    print("{} has {} frames".format(object_name, len(grasp_point_index)))

    pc = open3d.geometry.PointCloud()
    pc.points = open3d.utility.Vector3dVector(cloud)

    score = np.zeros([cloud.shape[0]])
    color = np.zeros([cloud.shape[0], 3])

    for i in grasp_point_index:
        score[i] = 0.99

    for i in range(cloud.shape[0]):
        color[i, :] = color_map(score[i])[0:3]

    pc.colors = open3d.utility.Vector3dVector(color)

    #while True:
    from utils.visualization_utils import get_hand_geometry
    #vis = open3d.visualization.VisualizerWithEditing()
    #vis.create_window()
    #vis.add_geometry(pc)
    #vis.run()
    #vis.destroy_window()
    #pick_inds = vis.get_picked_points()
    vis_list = [pc]
    for ind in grasp_point_index: #pick_inds:
        if ind in grasp_point_index:
            frame_index = grasp_point_index.index(ind)
            grasp = np.linalg.inv(grasp_pose[frame_index])
            #print(grasp_pose[frame_index])
            hand = get_hand_geometry(grasp)
            ball = open3d.geometry.TriangleMesh.create_sphere(0.0015)
            ball.translate(cloud[ind, :])
            vis_list.extend(hand)
            vis_list.append(ball)
            #break
    open3d.visualization.draw_geometries(vis_list)
    #

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/tmp/ipykernel_797238/3562786532.py:11: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = cm.get_cmap('jet', 256)


Lager has 72 frames


In [12]:
print(data.keys())
print(data['normal'].shape)

dict_keys(['grasp_pose', 'grasp_point_index', 'cloud', 'normal'])
(587, 3)


In [17]:
# create dummy data
import numpy as np
import pickle

POINT_CLOUD_SIZE = 5000
NUM_OBJECTS = 10

# Example data dictionary (replace with your actual data)
data_dict = {
    "point_cloud": np.random.rand(3, POINT_CLOUD_SIZE),  # 3D points
    "search_score": np.random.rand(NUM_OBJECTS,),
    "antipodal_score": np.random.rand(NUM_OBJECTS, ),
    "valid_frame": np.random.rand(NUM_OBJECTS, 4, 4),  # Transformation matrices
    "valid_index": np.random.choice(POINT_CLOUD_SIZE, size=NUM_OBJECTS, replace=False).astype(int),
    "direction": np.random.rand(NUM_OBJECTS, 5),
    "objects_label": np.zeros((NUM_OBJECTS,)).astype(int),
}

# Save to pickle file
output_file = "/home/sbreuers/vathos/s4g-release/tdgpd/datasets/bearring/val/scene_001.p"
with open(output_file, "wb") as f:
    pickle.dump(data_dict, f)

In [ ]:
#dict_keys(['search_score', 'antipodal_score', 'valid_frame', 'valid_index', 'point_cloud', 'objects_label'])

#point cloud
#(3, 48902)
#[[ 0.23269033  0.22019997  0.22615957  0.19583142  0.1853585 ]
# [ 0.34287396  0.3386026   0.33791447  0.34177426  0.34445158]
# [-1.014849   -1.0107541  -1.0115755  -1.0424486  -1.0506148 ]]

#search score
#(306,)
#[456.6667  387.66666 316.      294.      462.     ]

#antipodal_score
#(306,)
#[0.97540945 0.97183985 0.98422176 0.97475517 0.9976949 ]

#valid_frame
#(306, 4, 4)
#[[-0.21604782 -0.19880381  0.95592916 -0.24223596]
# [-0.8274021   0.55708593 -0.07114293  0.2534126 ]
# [-0.5183912  -0.80630785 -0.28484792 -1.2580454 ]
# [ 0.          0.          0.          1.        ]]

#valid_index
#(306,)
#[2389 2928 3564 3687 3688]

#objects_label
#(306,)
#[105 105  70  70  70]

In [8]:
np.random.choice(5, size=5, replace=False).astype(int)

array([0, 4, 1, 3, 2])